In [1]:
# autoreload import your package
%load_ext autoreload
%autoreload 2

import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')


## Debug model components

### Using trainer? :poop:

Hyrda is really annoying




In [2]:

import os
os.environ['WANDB_MODE'] = 'disabled'

import hydra
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

from pathlib import Path
from datetime import datetime

from src.trainer import Trainer


class Trainer2(Trainer):
    
    def load_checkpoint(self, *args, **kwargs):
        pass



ts = datetime.now().strftime("%Y-%m-%d/%H-%M-%S")
run_dir = Path(f"..outputs/{ts}").absolute()
run_dir.mkdir(parents=True, exist_ok=True)
abs_config_dir=os.path.abspath("../config")
os.chdir(run_dir)
# with initialize_config_dir(version_base=None, config_dir=abs_config_dir):
with initialize(version_base=None, config_path="../config"):
    cfg = compose(config_name='trainer', overrides=[
        f'hydra.run.dir={run_dir}',
        # f"initialization.path_to_checkpoint={str(path_to_checkpoint.absolute())}",
        'wandb.mode=disabled',
        "env.train.id=CrafterReward-v1",
        "training.tokenizer.start_after_epochs=1",
        "training.world_model.start_after_epochs=1",
        "training.actor_critic.start_after_epochs=1",
        "training.tokenizer.steps_per_epoch=10",
        "training.world_model.steps_per_epoch=10",
        "training.actor_critic.steps_per_epoch=10",
        "common.do_checkpoint=False",
        "common.resume=True",
        "training.world_model.batch_num_samples=4",
        "training.actor_critic.batch_num_samples=4",
        ])
    print(cfg)

    with run_dir:
        Path('media/episodes/train').mkdir(parents=True, exist_ok=True)
        trainer = Trainer2(cfg)
    trainer


/media/wassname/SGIronWolf/projects5/worldmodels/iris_bigvae/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


{'wandb': {'mode': 'disabled', 'project': 'iris', 'entity': None, 'name': None, 'group': None, 'tags': None, 'notes': None}, 'initialization': {'path_to_checkpoint': None, 'load_tokenizer': False, 'load_world_model': False, 'load_actor_critic': False}, 'common': {'epochs': 600, 'device': 'cuda:0', 'do_checkpoint': False, 'seed': 0, 'sequence_length': '${world_model.max_blocks}', 'resume': True}, 'collection': {'train': {'num_envs': 1, 'stop_after_epochs': 500, 'num_episodes_to_save': 10, 'config': {'epsilon': 0.01, 'should_sample': True, 'temperature': 1.0, 'num_steps': 200, 'burn_in': '${training.actor_critic.burn_in}'}}, 'test': {'num_envs': 8, 'num_episodes_to_save': '${collection.train.num_episodes_to_save}', 'config': {'epsilon': 0.0, 'should_sample': True, 'temperature': 0.5, 'num_episodes': 16, 'burn_in': '${training.actor_critic.burn_in}'}}}, 'training': {'should': True, 'learning_rate': 0.0001, 'tokenizer': {'batch_num_samples': 128, 'grad_acc_steps': 1, 'max_grad_norm': 10.0,

/media/wassname/SGIronWolf/projects5/worldmodels/iris_bigvae/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/media/wassname/SGIronWolf/projects5/worldmodels/iris_bigvae/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Using pad_token, but it is not set yet.


trainable params: 50,462,720 || all params: 1,150,511,104 || trainable%: 4.386113252149889
None
32314243 parameters in agent.tokenizer
752979973 parameters in agent.world_model
3224626 parameters in agent.actor_critic



## Trainer train_agent


In [3]:
self=trainer
epoch = 52

# get out first exp
self.train_collector.collect(self.agent, epoch, **self.cfg.collection.train.config)


Experience collection (train_dataset): 100%|██████████| 200/200 [00:03<00:00, 58.00it/s]


[{'train_dataset/episode_length': 183,
  'train_dataset/episode_return': tensor(0.1000),
  'train_dataset/episode_num': 0,
  'train_dataset/action_histogram': <wandb.sdk.data_types.histogram.Histogram at 0x7f97f7373eb0>},
 {'train_dataset/#episodes': 2,
  'train_dataset/#steps': 200,
  'train_dataset/return': 0.100000024}]

In [4]:
self.agent.train()
self.agent.zero_grad()

metrics_tokenizer, metrics_world_model, metrics_actor_critic = {}, {}, {}

cfg_tokenizer = self.cfg.training.tokenizer
cfg_world_model = self.cfg.training.world_model
cfg_actor_critic = self.cfg.training.actor_critic

# if epoch > cfg_tokenizer.start_after_epochs:
#     metrics_tokenizer = self.train_component(self.agent.tokenizer, self.optimizer_tokenizer, sequence_length=1, sample_from_start=True, **cfg_tokenizer)
# self.agent.tokenizer.eval()

# if epoch > cfg_world_model.start_after_epochs:
#     metrics_world_model = self.train_component(self.agent.world_model, self.optimizer_world_model, sequence_length=self.cfg.common.sequence_length, sample_from_start=True, tokenizer=self.agent.tokenizer, **cfg_world_model)
# self.agent.world_model.eval()

# if epoch > cfg_actor_critic.start_after_epochs:
#     metrics_actor_critic = self.train_component(self.agent.actor_critic, self.optimizer_actor_critic, sequence_length=1 + self.cfg.training.actor_critic.burn_in, sample_from_start=False, tokenizer=self.agent.tokenizer, world_model=self.agent.world_model, **cfg_actor_critic)
# self.agent.actor_critic.eval()


In [5]:
import torch
from torchinfo import summary
import torch
from einops import rearrange


## Directly benchmark models

In [6]:
tokenizer = self.agent.tokenizer
world_model = self.agent.world_model
actor_critic = self.agent.actor_critic


In [7]:
batch_num_samples = cfg.training.world_model.batch_num_samples
sequence_length = cfg.common.sequence_length
sample_from_start = False
# train_dataset = instantiate(cfg.datasets.train)
batch_num_samples


4

In [8]:
batch = self.train_dataset.sample_batch(batch_num_samples, sequence_length, sample_from_start)
batch = {k: v.to(self.device) for k, v in batch.items()}


In [9]:
%%time
self.agent.world_model.compute_loss(batch,  tokenizer=self.agent.tokenizer)


CPU times: user 190 ms, sys: 4.87 ms, total: 194 ms
Wall time: 195 ms


In [10]:
%%time
# TODO: why is this so slow?
cfg_actor_critic = self.cfg.training.actor_critic
self.agent.actor_critic.compute_loss(batch,  tokenizer=self.agent.tokenizer, world_model=self.agent.world_model, **cfg_actor_critic)


CPU times: user 9.35 s, sys: 17.3 ms, total: 9.37 s
Wall time: 9.37 s


In [11]:
%%time
# is this the slow part... yes. damn
actor_critic.imagine(batch, tokenizer, world_model, horizon=10);


CPU times: user 9.11 s, sys: 15.5 ms, total: 9.13 s
Wall time: 9.13 s


In [12]:
# # takes 0.1 s, fast
# wm_env = WorldModelEnv(tokenizer, world_model, device)
# wm_env


In [13]:
%%time
# this takes 0.1 seconds and is run 10+ time. So 1 second. Hmm
from src.envs.world_model_env import WorldModelEnv, Categorical
initial_observations = batch['observations']

# get the right obs
wm_env = WorldModelEnv(self.agent.tokenizer, self.agent.world_model, self.device)
obs = wm_env.reset_from_initial_observations(initial_observations[:, -1])
print(obs.shape)


# make sure hidden states are right
self.agent.actor_critic.reset(obs.shape[0])



torch.Size([4, 3, 64, 64])
CPU times: user 105 ms, sys: 207 µs, total: 105 ms
Wall time: 105 ms


In [14]:
import gc
gc.collect()
torch.cuda.empty_cache()
# obs


In [15]:
%%time
# 700us
# fast, executed 10+ times
outputs_ac = actor_critic(obs)


CPU times: user 1.6 ms, sys: 309 µs, total: 1.9 ms
Wall time: 1.72 ms


In [16]:
outputs_ac.logits_actions.shape
# action_token.shape


torch.Size([4, 1, 17])

In [17]:
# %%timeit
# slow! takes 1s, executed 10+ times this is the culprit, not the lstm. hmm
k=3
horizon = 6
action_token = Categorical(logits=outputs_ac.logits_actions).sample()
obs, reward, done, _ = wm_env.step(action_token, should_predict_next_obs=(k < horizon - 1))


In [18]:
%%timeit
# 62ms
# this is the slow part again. no grad and eval don't hepl
outputs_wm = world_model(action_token, past_keys_values=wm_env.keys_values_wm)


66.5 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:

num_steps=1
prev_steps=0
sequences = world_model.embedder(action_token, num_steps, prev_steps) + world_model.pos_emb(prev_steps + torch.arange(num_steps, device=action_token.device))


In [20]:
%%timeit
# ofc it's the transformer that's slow. I guess we just call it was more than during training
past_keys_values = wm_env.keys_values_wm
x = world_model.transformer(sequences, past_keys_values)


AssertionError: 

In [ ]:
# past_keys_values = wm_env.keys_values_wm
# x = world_model.transformer(sequences, past_keys_values)


In [ ]:
%%timeit
# ofc it's the transformer that's slow. I guess we just call it was more than during training
past_keys_values = wm_env.keys_values_wm
x = world_model.transformer(sequences, past_keys_values)


In [ ]:
logits_observations = world_model.head_observations(x, num_steps=num_steps, prev_steps=prev_steps)
logits_rewards = world_model.head_rewards(x, num_steps=num_steps, prev_steps=prev_steps)
logits_ends = world_model.head_ends(x, num_steps=num_steps, prev_steps=prev_steps)


## Torchinfo model sizes


In [ ]:
observations = self.agent.tokenizer.preprocess_input(rearrange(batch['observations'], 'b t c h w -> (b t) c h w'))
# z, z_quantized, reconstructions = self.agent.tokenizer(observations, should_preprocess=False, should_postprocess=False)
summary(self.agent.tokenizer, input_data=observations)


In [ ]:


with torch.no_grad():
    obs_tokens = self.agent.tokenizer.encode(batch['observations'], should_preprocess=True).tokens  # (BL, K)

act_tokens = rearrange(batch['actions'], 'b l -> b l 1')
tokens = rearrange(torch.cat((obs_tokens, act_tokens), dim=2), 'b l k1 -> b (l k1)')  # 

summary(self.agent.world_model, input_data=tokens)


In [ ]:
from src.envs.world_model_env import WorldModelEnv
initial_observations = batch['observations']

# get the right obs
wm_env = WorldModelEnv(self.agent.tokenizer, self.agent.world_model, self.device)
obs = wm_env.reset_from_initial_observations(initial_observations[:, -1])
obs.shape


# make sure hidden states are right
self.agent.actor_critic.reset(obs.shape[0])


In [ ]:


from torchinfo import summary
summary(self.agent.actor_critic, input_data=obs)


## Debug env


In [ ]:
import minihack
env = gym.make("MiniHack-River-v0", observation_keys=("pixel_crop", "pixel", 'blstats', 'message'))
env.reset() # each reset generates a new environment instance
obs, reward, end, info = env.step(1)  # move agent '@' north
print(obs['pixel_crop'].shape)
plt.imshow(obs['pixel_crop'])
plt.show()

print(obs['pixel'].shape)
plt.imshow(obs['pixel'])



In [ ]:
# # plt.imshow(obs['glyphs_crop'])
# obs['glyphs_crop'].shape
# obs['blstats']


In [ ]:
import minihack
env = gym.make("MiniHack-Room-5x5-v0", observation_keys=("pixel_crop", "pixel", 'blstats', 'message'))
env.reset() # each reset generates a new environment instance
obs, reward, end, info = env.step(1)  # move agent '@' north
print(obs['pixel_crop'].shape)
plt.imshow(obs['pixel_crop'])
plt.show()

print(obs['pixel'].shape)
plt.imshow(obs['pixel'])



In [ ]:
import minihack
import crafter
env = gym.make("CrafterReward-v1")
env.reset() # each reset generates a new environment instance
obs, reward, end, info = env.step(1)  # move agent '@' north
print(obs.shape)
plt.imshow(obs)
plt.show()
